In [12]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE

#### 加载自制中文数据集、分词、去除停用词

In [13]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['谣言'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['真相'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# jieba分词
import jieba
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_12357/1622170239.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


In [14]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize)
tfidf_docs = tfidf.fit_transform(raw_documents=df.text).toarray()
len(tfidf.vocabulary_)

/Users/stevenyu/opt/anaconda3/envs/nlpiaenv/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


4021

In [15]:
tfidf_docs = pd.DataFrame(tfidf_docs)
#通过减去每个文档（词袋向量）的平均值来中心化文档
tfidf_docs = tfidf_docs - tfidf_docs.mean()
tfidf_docs.shape
df.rumor.sum()

330

#### 基于PCA的短消息语义分析

In [16]:
from sklearn.decomposition import PCA
pca = PCA(n_components=16)
pca_docs = pca.fit_transform(tfidf_docs)
pca_topic_vectors = pca.transform(tfidf_docs)
columns = ['topic{}'.format(i) for i in range(pca_docs.shape[1])]
pca_topic_vectors = pd.DataFrame(pca_topic_vectors, columns=columns, index=index)
pca_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,...,topic12,topic13,topic14,topic15
rumor0,-0.046,0.056,-0.012,-0.014,...,-0.103,0.005,0.042,0.109
rumor1,-0.059,-0.039,0.028,0.001,...,-0.019,-0.008,0.002,-0.041
rumor2,0.032,0.030,-0.042,0.009,...,-0.020,-0.046,0.022,-0.065
rumor3!,-0.022,0.108,-0.050,0.021,...,-0.090,0.016,0.064,-0.094
rumor4!,-0.035,-0.078,-0.025,0.079,...,-0.079,0.104,-0.048,0.185
rumor5,0.166,-0.035,0.036,-0.107,...,-0.019,-0.010,-0.039,0.015


In [17]:
tfidf.vocabulary_
#根据词项的频率对词汇表进行排序
#当对某个不按照最左边元素排序的序列解压并在排序后重新压缩时，可以使用zip(*sorted(zip(...)))
column_nums , terms = zip(*sorted(zip(tfidf.vocabulary_.values(), tfidf.vocabulary_.keys())))


In [18]:
weights = pd.DataFrame(pca.components_, columns=terms,
                       index = ['topic{}'.format(i) for i in range(pca.components_.shape[0])])
pd.options.display.max_columns = 8
weights.head(4).round(3)

,%,0,0t,1,...,鼓励,鼻出血,鼻翼,龋齿
topic0,-0.001,-0.001,-0.004,-0.003,...,-0.001,-0.004,-0.002,-0.003
topic1,-0.003,-0.000,-0.002,-0.011,...,-0.002,-0.008,-0.002,-0.004
topic2,-0.001,0.000,0.001,-0.009,...,-0.001,-0.001,0.000,-0.001
topic3,0.001,-0.000,0.003,0.007,...,-0.003,-0.000,0.001,0.002


#### 基于截断的SVD的短消息语义分析（recommend）

In [19]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=16,n_iter=100)
svd_topic_vectors = svd.fit_transform(tfidf_docs.values)
svd_topic_vectors = pd.DataFrame(svd_topic_vectors, columns=columns, index=index)
svd_topic_vectors.round(3).head(6)

,topic0,topic1,topic2,topic3,...,topic12,topic13,topic14,topic15
rumor0,-0.046,0.056,-0.011,-0.015,...,-0.086,0.060,0.043,-0.087
rumor1,-0.060,-0.038,0.029,0.002,...,-0.019,-0.012,-0.031,0.026
rumor2,0.031,0.030,-0.042,0.009,...,-0.025,-0.047,-0.055,-0.026
rumor3!,-0.022,0.108,-0.052,0.020,...,-0.023,0.075,-0.107,-0.037
rumor4!,-0.035,-0.078,-0.026,0.077,...,-0.016,0.096,0.140,-0.031
rumor5,0.166,-0.035,0.036,-0.106,...,-0.035,0.004,0.014,0.040


#### 基于LSA的垃圾短消息分类的效果(PCA/SVD+LDA)

In [20]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(svd_topic_vectors,df.rumor,test_size=0.5,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA(n_components=1)
lda = lda.fit(X_train_res, y_train_res)
df['LSA16_rumor'] = lda.predict(svd_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 234
过采样前训练集中类别1的数量： 165
过采样后训练集中类别0的数量： 234
过采样后训练集中类别1的数量： 234


0.667

In [21]:
import numpy as np
svd_topic_vectors = (svd_topic_vectors.T / np.linalg.norm(svd_topic_vectors,axis=1)).T
svd_topic_vectors.iloc[:10].dot(svd_topic_vectors.iloc[:10].T).round(1)

,rumor0,rumor1,rumor2,rumor3!,...,rumor6!,rumor7!,rumor8,rumor9
rumor0,1.0,0.2,0.2,0.2,...,-0.1,0.3,0.1,-0.6
rumor1,0.2,1.0,0.2,-0.1,...,-0.5,0.4,0.2,0.4
rumor2,0.2,0.2,1.0,0.4,...,0.1,0.7,0.3,-0.1
rumor3!,0.2,-0.1,0.4,1.0,...,0.3,-0.3,0.2,-0.0
rumor4!,0.4,-0.0,-0.2,-0.0,...,-0.3,-0.1,0.1,-0.2
rumor5,-0.2,-0.4,-0.2,-0.2,...,0.6,-0.2,-0.2,-0.2
rumor6!,-0.1,-0.5,0.1,0.3,...,1.0,-0.2,-0.1,-0.2
rumor7!,0.3,0.4,0.7,-0.3,...,-0.2,1.0,0.1,-0.1
rumor8,0.1,0.2,0.3,0.2,...,-0.1,0.1,1.0,-0.1
rumor9,-0.6,0.4,-0.1,-0.0,...,-0.2,-0.1,-0.1,1.0


In [22]:
from sklearn.model_selection import cross_val_score
scores = cross_val_score(lda, svd_topic_vectors, df.rumor, cv=5)
"Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2)

'Accuracy: 0.70 (+/- 0.03)'